# Theta Function Animation Demo

This notebook demonstrates kaleidocycle animation using exact analytic solutions from Jacobi theta functions.

Please look at the following paper for details:
- Shizuo Kaji, Kenji Kajiwara, Shota Shigetomi,
  [*An explicit construction of Kaleidocycles*](https://arxiv.org/abs/2308.04977)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

import sys
sys.path.insert(0, '../src')

from kaleidocycle import (
    Kaleidocycle,
    KaleidocycleAnimation,
    plot_band,
    generate_theta_curve,
    generate_animation_theta,
    generate_theta_binormals,
    solve_closure_conditions,
    binormals_to_tangents,
    curve_to_binormals,
    tangents_to_curve,
    pairwise_cosines,
    compute_torsion,
    writhe,
    total_twist,
    format_report,
)

%matplotlib widget
plt.rcParams['figure.figsize'] = (12, 8)

## Closure Condition Solver

In [ ]:
# Import closure condition solver
from kaleidocycle import solve_closure_conditions
from kaleidocycle.theta import close1, close2

# Test cases: N values for different kaleidocycles
# Initial guess (N, m, (v, r)) only - y is computed from y = N*v/m
test_cases = [
    (6, 3, (0.64, 0.27)),
    (7, 3, (0.48, 0.27)),
    (8, 3, (0.40, 0.72)),
    (9, 3, (0.34, 0.29)),
    (15, 3, (0.19, 0.30)),
    (15, 5, (0.40, 0.69)),
#    (38, 3, (0.07, 0.30)),
]

print("="*80)
print("CLOSURE CONDITION SOLVER: Finding Parameters for Different N")
print("="*80)
print()

solutions = []

for N, m, initial_guess in test_cases:
    print(f"N = {N} tetrahedra with m = {m}:")
    print(f"  Solving with initial guess: v={initial_guess[0]:.2f}, r={initial_guess[1]:.2f}")

    result = solve_closure_conditions(N, m=m, initial_guess=initial_guess)

    if result:
        v, r, y = result
        solutions.append((N, m, v, r, y))

        print(f"  ✓ Solution found:")
        print(f"    v = {v:.10f}")
        print(f"    r = {r:.10f}")
        print(f"    y = {y:.10f}")

        # Verify closing conditions with new signature: close1(v, r, k, m)
        c1 = close1(v, r, N, m)
        c2 = close2(v, r, N, m)

        print(f"  Verification:")
        print(f"    |close1| = {abs(c1):.2e}")
        print(f"    |close2| = {abs(c2):.2e}")

        # Generate and check binormals
        try:
            #binormals = generate_theta_binormals(v, 0, r, y, N, t=0.0)
            curve = generate_theta_curve(v, 0, r, y, N, t=0.0)
            kc=Kaleidocycle(curve=curve)
            cosines = pairwise_cosines(kc.hinges)
            torsion_vals = compute_torsion(kc.hinges)

            print(f"  Kaleidocycle properties:")
            print(f"    Constant torsion: mean(cos)={np.mean(cosines):.8f}, std={np.std(cosines):.2e}")
            print(f"    Mean torsion angle: {np.degrees(np.mean(torsion_vals)):.4f}°")
        except Exception as e:
            print(f"    Warning: {e}")
    else:
        print(f"  ✗ No solution found")

    print()


## Visualize


In [ ]:
# Visualize kaleidocycles for N = 6, 8, 9, 13
fig = plt.figure(figsize=(20, 5))

for subplot_idx, (N,m,v,r,y) in enumerate(solutions, start=1):
        # Generate curve
        curve = generate_theta_curve(v, 0, r, y, N, t=0.0)
        kc = Kaleidocycle(curve=curve)

        # Plot
        ax = fig.add_subplot(1, len(solutions), subplot_idx, projection='3d')
        kc.plot(ax=ax, title=f'N = {N} Tetrahedra\nv={v:.4f}, r={r:.4f}, y={y:.4f}')

plt.tight_layout()
plt.show()


## Animated Visualization


In [ ]:
from kaleidocycle import plot_band, constraint_penalty, ConstraintConfig, mean_cosine, export_csv
from matplotlib import animation

# Generate animation with theta functions
num_frames = 50
t_step = 0.05
z=0
N,m,v,r,y=solutions[5]
oriented = m % 2 == 0

frames = generate_animation_theta(v, z, r, y, N, num_frames, t_step, output="curve")
anim = KaleidocycleAnimation.from_curves(frames)

print(f"Generated {len(frames)} frames")
print(f"Each frame shape: {frames[0].shape}")
print(f"N, m, v, r, y = {N}, {m}, {v:.6f}, {r:.6f}, {y:.6f}")

f = plot_band(animation=anim)
plt.show()


In [ ]:
anim.compute_scalar_property("penalty")

plt.close('all')
plt.plot(anim.scalar_properties["penalty"], marker='o')
plt.xlabel("Frame")
plt.ylabel("Penalty")
plt.title("Penalty Evolution")
plt.grid()
plt.show()